In [ ]:
import sys
import time
import pandas as pd
from sklearn import model_selection
from sklearn import ensemble, linear_model, svm, metrics, cluster, tree
from xgboost import XGBRegressor
from matplotlib import pyplot as plt
import scipy
import os
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.feature_selection import *
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import GridSearchCV, KFold

start_time = time.time()

n_cv = 5
n_jobs = 6
random_state = 3111696
d_set = "Galaxy215-[rTRAIL.csv].tabular"

print("Processing dataset: %s" % d_set)
df = pd.read_csv("depmap/" + d_set, sep="\t")
columns = list(df.columns.values)
label = df[columns[-1]].copy()
data = df.drop(columns[-1], axis=1)
print(data.shape)
print(label.shape)

print('Training started...')

pipe_regressor = Pipeline([
  ('regression', XGBRegressor())
])

parameters = [
    {   
        'regression__n_estimators': [50, 100],
        'regression__random_state': [random_state]
    }
]

optimized_regressor = GridSearchCV(pipe_regressor, parameters, \
                                       cv=KFold(n_splits=n_cv, shuffle=True, random_state=random_state), \
                                       error_score=0, scoring='r2', verbose=True, n_jobs=n_jobs, \
                                       pre_dispatch="1*n_jobs")
optimized_regressor.fit(data, label)
best_regressor = optimized_regressor.best_estimator_
best_result = optimized_regressor.cv_results_
print(optimized_regressor.best_params_)
best_score = optimized_regressor.best_score_
print(best_score)
f_importance = optimized_regressor.best_estimator_.named_steps["regression"].feature_importances_
indices = np.argsort(f_importance)[::-1]

print('Training finished')
print("R2 square: \n", best_score)

Processing dataset: Galaxy215-[rTRAIL.csv].tabular


In [ ]:
topk = 200
topk_indices = indices[:topk]

plt.figure(figsize=(13, 5))
plt.title("Feature importances")
plt.bar(range(topk), f_importance[topk_indices], color="r")
plt.xlabel("Number of features")
plt.ylabel("Importance")
plt.grid(True)
plt.show()

end_time = time.time()
print('Total time taken: %d seconds' % int(end_time - start_time))